In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [2]:
# отмасштабируем один из признаков
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

#### Задание 1: 
*Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.*

In [3]:
import numpy as np

# Я не уверен, правильно ли я понял задание, но избежать мы можем просто сместив значение на минимальную величину
# Введем параметр minEps, при y_pred==0 заменим y_pred на minEps, а при y_pred==1 на (1-minEps)
# Естественно, подразумевается, что minEps примерно равна 0 и отличается от нее меньше значимого порядка предсказания

def calc_logloss(y, y_pred, minEps = 1e-8):
    
    y_pred = np.where(y_pred==0, minEps, y_pred) 
    y_pred = np.where(y_pred==1, 1-minEps, y_pred) 

    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

#### Задание 2: 
*Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.*

In [4]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot(X, (y_pred - y).T))
    print(i, W, err)
    return W

In [5]:
W = eval_model(X_st, y, iterations=30000, alpha=40)

30000 [-292.17003934  -20.04019167  -87.04368153  222.4874873 ] 0.0023089657288500104


Возможно это ошибка, но функция получилась убывающей. Мы можем до бесконечности приближаться к нулю увеличивая скорость обучения и количество иттераций.

Остановился на скорости 40 (т.к. при большей скорости python не можемт посчитать exp) и количестве иттераций - 30000, чтобы время поиска было адекватное.

#### Задание 3: 
*Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).*

In [6]:
def calc_pred_proba(w, X):
   
    w = w.reshape(X.shape[0], 1)
    y_pred_proba = sigmoid(np.dot(w.T, X))
    
    return y_pred_proba

In [7]:
y_pred_proba = calc_pred_proba(W, X_st)
y_pred_proba

array([[1.14794522e-02, 2.97114394e-18, 1.00000000e+00, 1.85750323e-10,
        9.99399572e-01, 2.40028309e-17, 1.00000000e+00, 1.10885085e-04,
        9.89226487e-01, 1.00000000e+00]])

#### Задание 4: 
*Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).*

In [8]:
def calc_pred(w, X):
    y_predicted = np.zeros((1, X.shape[1]))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X))
    
#     За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(A.shape[1]):
        if (A[:,i] > 0.5): 
            y_predicted[:, i] = 1
        elif (A[:,i] <= 0.5):
            y_predicted[:, i] = 0
    
    return y_predicted

In [9]:
y_pred = calc_pred(W, X_st)
y_pred

array([[0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]])

#### Задание 5: 
*Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.*

In [10]:
def accuracy(y, y_pred):
    return np.sum(np.equal(y, y_pred)) / y.shape[0]


accuracy(y, y_pred)

1.0

In [11]:
def errorMatrix(y, y_pred):
    Xor = np.logical_xor(y, y_pred)
    Eq = np.logical_not(Xor)
    
    PPred = y_pred
    NPred = np.logical_not(y_pred)

    TP = np.sum(np.logical_and(PPred, Eq))
    TN = np.sum(np.logical_and(NPred, Eq))
    FP = np.sum(np.logical_and(PPred, Xor))
    FN = np.sum(np.logical_and(NPred, Xor))
    
    return TP, FP, FN, TN

TP, FP, FN, TN = errorMatrix(y, y_pred)

df = pd.DataFrame({
    "y=+1": [TP, FP],
    "y=-1": [FN, TN],
    "type": ['pred=+1', 'pred=-1']
})
df.set_index('type')

,y=+1,y=-1
type,,
pred=+1,5,0
pred=-1,0,5


#### Задание 6: 
*Могла ли модель переобучиться? Почему?*

Да, модель скорее всего переобучилась, мы довели ошибку до минимума и в результате получили довольно высокие коэффициенты по первому и последнему признакам.

А так как мы предварительно не отсеяли выбросы, то есть высокий риск, что именно они могли "испортить" нашу модель и привести к переобучению.

Поэтому на проверочных данных точность скорее всего будет крайне низкая.

Очевидно здесь нужно добавить регуляризацию.

Либо вывести график изменения ошибки в зависимости от увеличения количества иттераций для разных скоростей обучения и останавливать обучение не на минимуме фукнции ошибки, а на "сгибе", т.е. моменте, когда увеличение обучения перестает давать существенный эффект (т.е. увеличение иттераций дает незначительное уменьшение ошибки). Поидее именно этот момент будет отделять обученную модель от переобученной.
